In [1]:
import numpy as np
import pandas as pd
import os
import sys
import time

In [2]:
seeds = pd.read_csv(os.path.join(sys.path[0], '../Data/2020DataFiles/2020DataFiles/2020-Mens-Data/MDataFiles_Stage1/MNCAATourneySeeds.csv'))
tourneyCompactResults = pd.read_csv(os.path.join(sys.path[0], '../Data/2020DataFiles/2020DataFiles/2020-Mens-Data/MDataFiles_Stage1/MNCAATourneyCompactResults.csv'))
regSeasCompactResults = pd.read_csv(os.path.join(sys.path[0], '../Data/2020DataFiles/2020DataFiles/2020-Mens-Data/MDataFiles_Stage1/MRegularSeasonCompactResults.csv'))
regSeasDetailedResults = pd.read_csv(os.path.join(sys.path[0], '../Data/2020DataFiles/2020DataFiles/2020-Mens-Data/MDataFiles_Stage1/MRegularSeasonDetailedResults.csv'))


In [ ]:
print('')
print(seeds[0:5])
print(results[0:5])

In [ ]:
seeds['TeamID'] = pd.Categorical(seeds.TeamID)
seeds['section'] = seeds.Seed.str.extract(r'(.)')
seeds['numSeed'] = seeds.Seed.str.extract(r'([0-9][0-9])')
seeds['playInRank'] = seeds.Seed.str.extract(r'[0-9][0-9](.?)')

seeds['playInRank'].replace(to_replace = ['a', 'b', ''], value = [1, 2, 0], inplace = True)
seeds['playInRank'] = seeds.playInRank.astype(float)

seeds['numSeed'] = seeds.numSeed.astype(float)

seeds['numSeed'] = seeds.numSeed + (seeds.playInRank / 10)

seeds.drop(['Seed', 'playInRank'], axis = 1, inplace = True)

In [ ]:
seeds.to_csv('test.csv', index = False)

In [ ]:
results['WTeamID'] = pd.Categorical(results.WTeamID)
results['LTeamID'] = pd.Categorical(results.LTeamID)

In [ ]:
seeds.rename(columns = {'TeamID': 'WTeamID'}, inplace = True)
seedResults = pd.merge(results, seeds, on = ['WTeamID', 'Season'])
seedResults.rename(columns = {'section': 'WSection', 'numSeed': 'WNumSeed'}, inplace = True)

seeds.rename(columns = {'WTeamID': 'LTeamID'}, inplace = True)
seedResults = pd.merge(seedResults, seeds, on = ['LTeamID', 'Season'])
seedResults.rename(columns = {'section': 'LSection', 'numSeed': 'LNumSeed'}, inplace = True)

seeds.rename(columns = {'LTeamID': 'TeamID'}, inplace = True)

In [ ]:
#regSeasTotals1985 = pd.read_pickle(path = os.path.join(sys.path[0], '../GeneratedData/regSeasTotals1985.pkl'))
#seedResults1985 = seedResults[:]
print('rstFn')
print(regSeasTotals1985.head())
print('srFn')
print(seedResults1985.head())

In [ ]:
srFn = seedResults1985[:]
rstFn = regSeasTotals1985[:]

rstFn.rename(columns = {'TeamID': 'WTeamID'}, inplace = True)
outDF = pd.merge(srFn, rstFn, on = ['WTeamID', 'Season'])
outDF.rename(columns = {'W': 'WWins', 'L': 'WLosses', 'PF': 'WPF', 'PA': 'WPA'}, inplace = True)

rstFn.rename(columns = {'WTeamID': 'LTeamID'}, inplace = True)
outDF = pd.merge(outDF, rstFn, on = ['LTeamID', 'Season']) 
outDF.rename(columns = {'W': 'LWins', 'L': 'LLosses', 'PF': 'LPF', 'PA': 'LPA'}, inplace = True)

In [ ]:
outDF.head()

In [ ]:
nrow = seedResults.shape[0]
seedResults['WExpected'] = seedResults.WTeamID

start = time.time()

for i in range(nrow):
    if seedResults.loc[i, 'WNumSeed'] >= seedResults.loc[i, 'LNumSeed']:
        seedResults.loc[i, 'WExpected'] = seedResults.loc[i, 'LTeamID']

end = time.time()

print(end - start)

seedResults['correctGuess'] = seedResults.WExpected == seedResults.WTeamID

In [ ]:
accuracy = seedResults.correctGuess.sum() / nrow
print("Picking highest seed strategy accuracy: ", accuracy)

In [ ]:
start = time.time()

years = regSeasResults.Season.unique()
masterSeasonTotals = pd.DataFrame(columns = ['Season', 'TeamID', 'W', 'L', 'PF', 'PA'])

for year in years[0:2]:
    print(year)
    actSeason = regSeasResults[regSeasResults.Season == year]
    actSeason.reset_index(inplace = True)
    teams = actSeason.WTeamID.append(actSeason.LTeamID).unique()
    nrow = len(teams)
    actSeasonTotals = pd.DataFrame(data = np.zeros((nrow, 6)), columns = ['Season', 'TeamID', 'W', 'L', 'PF', 'PA'])

    actSeasonTotals['Season'] = pd.Series(data = [year] * nrow)
    actSeasonTotals['TeamID'] = pd.Series(data = teams)

    for game in range(actSeason.shape[0]):
        WInd = actSeasonTotals.index[actSeasonTotals['TeamID'] == actSeason.loc[game, 'WTeamID']].tolist()[0]
        LInd = actSeasonTotals.index[actSeasonTotals['TeamID'] == actSeason.loc[game, 'LTeamID']].tolist()[0]
        actSeasonTotals.loc[WInd, 'W'] += 1
        actSeasonTotals.loc[LInd, 'L'] += 1
        actSeasonTotals.loc[WInd, 'PF'] += actSeason.loc[game, 'WScore']
        actSeasonTotals.loc[WInd, 'PA'] += actSeason.loc[game, 'LScore']
        actSeasonTotals.loc[LInd, 'PF'] += actSeason.loc[game, 'LScore']
        actSeasonTotals.loc[LInd, 'PA'] += actSeason.loc[game, 'WScore']

    masterSeasonTotals = pd.concat([masterSeasonTotals, actSeasonTotals], axis = 0)
        
end = time.time()

print(end - start)

In [6]:
years = regSeasResults.Season.unique()
columns = ['Season', 'TeamID', 'G', 'W', 'L', 'PF', 'PA', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF']
masterSeasonTotals = pd.DataFrame(columns = columns)
ncol = len(columns)

for year in years:
    actSeason = regSeasResults[regSeasResults.Season == year]
    actSeason.reset_index(inplace = True)
    teams = actSeason.WTeamID.append(actSeason.LTeamID).unique()
    nrow = len(teams)
    actSeasonTotals = pd.DataFrame(data = np.zeros((nrow, ncol)), columns = columns)

    for game in range(actSeason.shape[0]):
        WInd = actSeasonTotals.index[actSeasonTotals['TeamID'] == actSeason.loc[game, 'WTeamID']].tolist()[0]
        actSeasonTotals.index[actSeasonTotals['TeamID'] == actSeason.loc[game, 'LTeamID']].tolist()[0]
        actSeasonTotals.loc[WInd, 'W'] += 1
        actSeasonTotals.loc[LInd, 'L'] += 1
        actSeasonTotals.loc[WInd, 'PF'] += actSeason.loc[game, 'WScore']
        actSeasonTotals.loc[WInd, 'PA'] += actSeason.loc[game, 'LScore']
        actSeasonTotals.loc[LInd, 'PF'] += actSeason.loc[game, 'LScore']
        actSeasonTotals.loc[LInd, 'PA'] += actSeason.loc[game, 'WScore']

        if ncol > 6:
            for j in columns[7:]:
                actSeasonTotals.loc[WInd, j] += actSeason.loc[game, ('W' + j)]
                actSeasonTotals.loc[LInd, j] += actSeason.loc[game, ('L' + j)]

    masterSeasonTotals = pd.concat([masterSeasonTotals, actSeasonTotals], axis = 0)
        

NameError: name 'regSeasResults' is not defined

In [7]:
jack = pd.DataFrame(columns = ['Season', 'TeamID', 'W', 'L', 'PF', 'PA'])
jack.shape

(0, 6)

In [13]:
regSeasDetailedResults.columns
columns = ['Season', 'TeamID', 'G', 'W', 'L', 'PF', 'PA', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF']
'W' + columns[7]

'WFGM'